## Lab 3. Extrapolation techniques for accurate thermochemistry



In this lab you will:

## Part A

- Calculate the bond dissociation energy (BDE) in the $F_2$ molecule with a large basis set and accurate correlated method (CCSD(T))

- Perform basis set extrapolation using an exponential or power series extrapolation to accunt for finite basis set effects (intrinsec basis set insufficientcy (IBSI)). 

- How to use the correlation energy additivity scheme.

## Part B

- In Part A, you made use of an exponential/power series extrapolation scheme to account for the finite basis set effects. In other words, you have accounted for the so called 'intrinsic basis set insufficiencie IBSI'. IN the second part, we will be correcting for another competing phenomenon called basis set superposition error (BSSE).


Authors: Lyudmila Slipchenko (lslipchenko@purdue.edu; ORCID: 0000-0002-0445-2990) and Victor H. Chavez (gonza445@purdue.edu; ORCID: 0000-0003-3765-2961),




---

In [ ]:
import psi4
import qcelemental as qcel
import numpy as np
import matplotlib.pyplot as plt

## Part A | Lab procedure:

### **1.** 
Optimize geometry of $F_2$ molecule at the *MP2/cc-pvdz* level. (Perform a Potential Energy Surface (PSE) similar to what you did in Lab 2). Plot the PES to confirm the optimal geometry. Write down the energy in kcal/mol. 



### **1.1** 
Calculate the zero-point vibrational energy (ZPE). 
The ZPE can be obtained directly from the output of a frequency calculation. You will need to run the script "freq.py" on this folder on the terminal as:

```
python freq.py 
```

**Note** For this to work, you need to be in the correct conda environment. In order to do this, while you are in your scholar terminar, run the following commands:

```
module use /depot/lslipche-class/apps/chm673/2020/modulefiles  
module load conda-env/chm673-py3.6.4
```

Look at the final lines of what's printed on the terminal ("Thermochemistry Energy Analysis") and write down the Correction ZPE for each of the methods(HF, MP2, CCSD(T)). 
You will need to modify the method and run the script each time. 
Write down the energies in kcal/mol


In [ ]:
#RESPONSE

Under the hood Psi4 obtains ZPE by adding all of the frequencies associated to the ground state. Proceeds to add them in the appropriate units. Remember how in class we saw that the ZPE corresponded to the energy between the bottom of the PES and the energy of the first state. 



---

### **2.** 
Using the found optimal geometry of $F_2$ molecule, calculate the HF, MP2 and CCSD(T) (coupled cluster with single and double excitations and perturbateive triple corrections method) energies in a series of the cc-pvdz, cc-pvtz and cc-pvqz bases. Write down all these energies. Write down the number of basis function in each basis. 

#### Hint 1: The HF and MP2 energies will be printed in the CCSD(T) output file calculations as well. So that you do not need to separate the calculations. On the other hand you may use a loop to get the energies and avoid having to go through the output file. 

#### Hint 2: You can quickly have access to the number of basis functions with"
```
wfn.basisset().nbf()
```

#### Hint 3: If you create a dictionary with keys as the basis, and then each of the keys contains a dictonary with the basis, you can fill in in a double loop and then display it nicely using pandas.  This will be usefull through the whole Lab. 

```
dictionary_energies = {"cc-pvdz":{"HF": 0.0, "MP2": 0.0, "CCSD(T)": 0.0, "basisfuncs":0.0}, ...}
#And then fill in with 
dictionary_energies['cc-pvdz']['HF'] = e_hf 

```
---

```
#You will display the energies as:
import pandas as pd
df_energies = pd.DataFrame(dictionary_energies)
df_energies
```

If you feel that you won't feel confortable using pandas. You may use excel to keep track of your data. 

---

## Part A | Lab Report:


### **1.** 
Estimate the basis set limits of the HF energies(i.e., complete basis set (CBS) HF energies) for $F_2$ and F by using the following formula:

$$ E^{HF}_{x} = E^{HF}_{CBS} + B e^{-Ax}$$

Where:  
**A**, **B** -> Constants to be determined.    
**$E_x$** is the HF energy in cc-pv$\mathbf{x}$z basis, specifically it corresponds to the highest angular momentum included in the basis set. 
And **E_{CBS}** is the sought-for energy in the complete basis set. 

The extrapolation scheme requires 3 basis; use the 2,3 and 4 basis to obtain the CBS values (i.e., X=2,3,4) 
#### Hint: Think of building a system of equations.

#### Hint the library scipy has a module that can be accessed as 

```
from scipy.optimize import fsolve
``` 

It can be used to find the solution of non-linear equations. Here is an example for the system:

$$ x^2 = -xy $$
$$ x^3 = -2z^2$$
$$ y^4 + z = 0$$

```
def equations(var):
    x,y,z = var
    
    f1 = x*2 + xy
    f2 = x**3 + 2z**2
    f3 = y**4 + z
    
    return (f1,f2,f3)
    
x,y,z = fsolve(equations,[x_0, y_0, z_0])

#Where [x_0, y_0, z_0] are the initial guesses for each of the unknowns. For a and b you can try values close to 1, and for E_(cbs) the HF energy. 
```



#### The equations that we need to solve are the following:

1. 
$$ E^{HF}_{2} = E^{HF}_{CBS} - Be^{-2A}$$  


2. 
$$ E^{HF}_{3} = E^{HF}_{CBS} - B e^{-3A}$$  


3. 
$$ E^{HF}_{4} = E^{HF}_{CBS} - B e^{-4A}$$  

With these equations, we have the required number of equations needed to obtain the 3 parameters.

In [ ]:
#RESPONSE

---

### **2.** 

Estimate the MP2 and CCSD(T) correlation energy basis set limits (i.e., E(MP2)-E(HF) and E(CCSD(T))-E(HF)) by the following formula:

$$E_x^{corr} = E_{CBS}^{corr} + CX^{-3}$$

Where:  
**C** is a coefficient we required to find.  
**$Ex$** and **$E_{CBS}$** are the correlation energies in the cc-pvxz basis and CBS limit.  

This is two-point scheme, you need two basis sets to obtain CBS energies. Calculate Mp2 and CCSD(T) CBS *correlation* energies using 1) X=2,3 and 2) X= 3,4 


In [ ]:
#RESPONSE

---

### **2.1** 

Calculate the MP2 and CCSD(T) total energy CBS limits, which are sums of the HF CBS energies and the correlation CBS energies. Note that you will have two different values of the CBS energies. Refer them as CBS(2,3) and CBSD(3,4). That is, in this step you calculate:

MP2/CBS(2,3)  
MP2/CBS(3,4)  
CCSD(T)/CBS(2,3)  
CCSD(T)/CBS(3,4) 

Energies of **$F_2$** and **$F$**

#### If you created your dictionary with basis sets and energies. You can use it again here to add the previous results in a new keys "CBS(2,3)" and CBS(3,4). 

e.g.

```
cbs_23 = {"CBS(2,3)": {"MP2":-1.0, 
                       "CCSD(T)" : -2.0}}

cbs_34 = {"CBS(3,4)": {"MP2":-1.5, 
                       "CCSD(T)" : -2.5}}
                       
dictionary_energies.update(cbs_23)
dictionary_energies.update(cbs_23)
 
```

In [ ]:
#RESPONSE

---

### **3** 

Calculate the bond dissociation energies:

$$ E_{diss} = E(F_2) - 2E(F) $$

By HF, MP2 and CCSD(T) in different basis sets.

#### Hint: If you have been following the Notebook with Pandas. This step should be effortless. Just substract the molecular dataframe minus two times the atom data frame. 


In [ ]:
#RESPONSE

---

### **3.1** 

Plot HF, MP2, and CCSD(T) bond dissociation energies vs the size of the basis set including CBS values. Find an experimental value of BDE in $F_2$ And add the experimental BDE to the plot and compare it with the calculated BDEs. 

#### Attention: When comparing experimental values and computational BDEs, you need to account for ZPE energy. You may either shift experimental BDE energy by computed ZPE or include ZPE in all computational values of BDEs

**Hint:** You can take your pandas data frame and have access to the rows/columns of data for easy ploting. This is done with the method "iloc". This will let you access the information with the values of the indices just like in a numpy array:

```
dictionary_energies.iloc[3,:].to_list()
```

In [ ]:
#RESPONSE

---

### **4** 

Analyze and discuss your results using the following questions as a guideline. Assume that the **targeted accuracy in prediction of BDE is 1kcal/mol**

a) Compare convergence of HF, MP2 and CCSD(T) wrt basis set. Why do the HF energies converge faster than the correlated methods?

b) Compare MP2 CBS extrapolations with X=2,3 and X=3,4. WHat is the difference between the two extrapoaltions? Is X=2,3 extrapolation reliable, i.e., is the difference between X=2,3 and X=3,4 extrapolations less than the targeted accuracy of 1 kcal/mol?

c) Do the same analysis for CCSD(T) extrapolation schemes. 

d) Discuss the accuracy and reliability of HF, MP2 and CCSD(T
 in CBS for predicting BDE. What can you say about accuracy and reliability of HF, MP2 and CCSD(T energies in the cc-pVTZ basis?
 
 e) Use the energy aditivity scheme for MP2 and CCSD(T) energies:
 
 $$ E_{big} (CCSD(T) = E_{small} (CCSD(T)) + [E_{big}(MP2) - E_{small}(MP2)] $$,
 
Where $E_{big}$ and $E_{small}$ are energies in bigger and smaller basis sets. Use cc-pVTZ and cc-pVQZ bases in this formula and compare the exact CCSD(T)/cc-pVQZ total energies of $F_2$ and F with the ones obtained by the formula. Calculate BDEs based on this extrapolation. What is the error in BE produced by this formula?

f) Based on your results, what is the **computationally cheapest way** to achieve the **target accuracy** in predicting BDE of $F_2$ (Let us assume that electronic structure calculations are the time-limiting step, rather than your calculations in excel/python). 

In [ ]:
#RESPONSE

---

## Part B

In part A, you made use of an exponential/power series extrapolation scheme to account for the finite basis set effects. In other words, you have accounted for the so called 'intrinsic basis set insufficiencie IBSI'. IN the second part, we will be correcting for another competing phenomenon called basis set superposition error (BSSE).

When trying to compute the interaction energy between dimer complexes, each monomer can technically 'borrow' (or in Prof. David Sherrill's words, 'steal') the basis functions from the other monomer. This causes an artificial stabilization of the dimer complex, an hence interaction energies are overestimated in most cases. To circumvent this issue, we can compute the energy of the monomers in the dier basis. This involves an extra step, where the first monommer is simulated in the presence of 'ghost' atoms (basis functions exist, but electrons are absent) of the second monomer, and the same is repeated for the second monomer. This is called the 'counterpoise' correction, formulated by Boys and Bernardi. Psi4 performes the entire process in one step using the following:

```
en = psi4.energy('MP2/aug-cc-pVDZ', molecule=mol, bsse_type='cp')
```

In this lab, we will be utilizing this scheme for the water dimer. 

### Part B | Lab procedure:

### 1. 
Below is the geometry for the water dimer we are interested in:
    
H     0.093  0.555  0.000  
O    -0.001  1.519  0.000   
O    -0.001 -1.389  0.000   
H     0.901  1.849  0.000   
H    -0.488 -1.720  0.761   
H    -0.488 -1.720 -0.761  

If you want Psi4 to perform the automatic energy correction, an extra step is required in the geometry definition. You will need to add a line containing '--' in between the two distinct fragments. E.g. in He2:


he_dimer = psi4.geometry("""  
He 0.0 0.0 0.0 

--  
He 0.0 0.0 0.0 3.75  
""")

**Note:** When you are using the bsse_type flag, the resulting energy (en) will correspond to the CP corrected interaction energy (in hartrees).   
Another option is to look at the output file. This appears at the end of the file with the title: "N-Body: Coutnerpoise Corrected (CP energies)".  
If you look at the output file, you will notice that many calculations were done. Identify the differences by looking at the geometries. They will help you solve some of the questions at the end. Whenever you see a Gh(atom), it means that only basis sets are added in that geometry. 
Analogously, you could run the BSSE calculation manually by performing the same calculations. If you want to try that. The ghost atoms in the definition of the geometry are specified with an "@" before the name of the atom. 

In [ ]:
#RESPONSE

---

### 2. 
Use three different basis sets: 3-21G (small), 6-31+G**(medium) and aug-cc-pVTZ(large) and two different methods: HF and P2. You will have to perform six different calculations in total. Give the resulting energies (CP-Corrected) in kJ/mol. (1 kcal/mol = 4.148 kJ/mol)

In [ ]:
#RESPONSE

### 2. Questions

1. In which basis sets (small vs medium vs large) do you observe BSSE the most? Can you  explain briefly why?  

2. In which method do you observe BSSE the most? in HF or in MP2? Can you thing of any reasoning behind the answer?  

3. What is the additional computational cost of performing the complete BSSE calculation? Possible answers are:  
    a. Same as dimer calculation cost.    
    b. 2 or 3 times the monomer calculation cost.  
    c. 2 or 3 times the dimer calculation cost.  
    d. An order higher than dimer calculation cost.  
Justify your answer.  

4. You made use of three different basis sets of varying sizes for these simulations. Take these three cases one by one, explain which error (IBSI vs BSSE) is dominant in each of them. Analyze and think carefully: The answer is not very straight forward!  

5. Compare the numbers you have obtained with the CCCBDB results. Do you observe similar trends? Whad do you think about DFT's performance in handling BSSE?  
(Don't fret on your your numbers are not matching exactly with CCCBDB. The dimer geometries are different and hence you should expect your numbers to be slightly different).  

6. *Bonus*: Assume you are working with a large molecule. Is it possible that one part of the molecule 'borrows' or 'steals' teh basis functions from the other part, resulting in some sort of BSSE within the molecule itself> Try searching the literature for answers. You get bonus points if:

    a. You answer the above question and explain why it exists.   
    b. Mention the exact term for this phenomenon.   
    c. How large s this effect? Is it okay to neglect them in routine electronic structure simulations, or do you have to account for them explicitly.   